In [1]:
# APPROACH TO ONLY TRAIN ON ONE AND TWO 

In [2]:
import datetime
import os
import pickle
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as stats
from sklearn.inspection import PartialDependenceDisplay, partial_dependence
from sklearn.metrics import (ConfusionMatrixDisplay, accuracy_score, precision_recall_curve, roc_auc_score, roc_curve)
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.preprocessing import label_binarize
from xgboost import XGBClassifier

import shap

%config InlineBackend.figure_format = 'retina'

In [3]:
from notebook import notebookapp
import urllib
import json
import ipykernel
from shutil import copy2

def notebook_path():
    """Returns the absolute path of the Notebook or None if it cannot be determined
    NOTE: works only when the security is token-based or there is also no password
    """
    connection_file = os.path.basename(ipykernel.get_connection_file())
    kernel_id = connection_file.split('-', 1)[1].split('.')[0]

    for srv in notebookapp.list_running_servers():
        try:
            if srv['token']=='' and not srv['password']:  # No token and no password, ahem...
                req = urllib.request.urlopen(srv['url']+'api/sessions')
            else:
                req = urllib.request.urlopen(srv['url']+'api/sessions?token='+srv['token'])
            sessions = json.load(req)
            for sess in sessions:
                if sess['kernel']['id'] == kernel_id:
                    return os.path.join(srv['notebook_dir'],sess['notebook']['path'])
        except:
            pass  # There may be stale entries in the runtime directory 
    return None


def copy_current_nb(new_name):
    nb = notebook_path()
    if nb:
        new_path = os.path.join(os.path.dirname(nb), new_name+'.ipynb')
        copy2(nb, new_path)
    else:
        print("Current notebook path cannot be determined.")

In [4]:
df = pd.read_csv('Data/cover_type_engineered.csv')

In [16]:
df = df.loc[:, [col for col in df if not col.startswith('Cover_Type_')]]
df = df.loc[(df['Cover_Type'] == 1) | (df['Cover_Type'] == 2)]

X = df.drop(columns=['Cover_Type', 'Aspect_Sector'])
y = df['Cover_Type'] - 1

In [ ]:
timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M")

warnings.filterwarnings("ignore", category=FutureWarning, module="pandas.api.types")

# Assuming X and y are defined
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.33, random_state=42)

# Define the estimator
estimator = XGBClassifier(tree_method="hist")

# Define hyperparameters for tuning
hyperparameters = {
    "n_estimators": stats.randint(50, 150),  # Increased range
    "learning_rate": stats.uniform(0.05, 0.1),  # Adjusted range
    "colsample_bytree": stats.uniform(0.6, 0.4),
    "colsample_bylevel": stats.uniform(0.6, 0.4),
    "colsample_bynode": stats.uniform(0.6, 0.4),
    "max_depth": stats.randint(15, 50),  # Increased range
    "subsample": stats.uniform(0.6, 0.4),
    "gamma": stats.uniform(0, 2),  # Adjusted range
    "reg_lambda": stats.uniform(0, 2),  # Adjusted range
    "reg_alpha": stats.uniform(0, 2),  # Adjusted range
}

# Define the RandomizedSearchCV parameters
random_search = RandomizedSearchCV(estimator, 
                                   param_distributions=hyperparameters, 
                                   scoring='accuracy',
                                   return_train_score=True,
                                   n_iter=500, 
                                   cv=5, 
                                   verbose=10, 
                                   n_jobs=-1)

# Fit the RandomizedSearchCV
try:
    random_search.fit(X_train, y_train)  # Assuming X_train and y_train are defined
    print("Best parameters found:", random_search.best_params_)
    print("Best score found:", random_search.best_score_)

    
    # Save results
    results_path = f"./tuning_results/tuning_xgb/{timestamp}"
    if not os.path.exists(results_path):
        os.makedirs(results_path)
        
    # Saving cross-validation results
    cv_results = pd.DataFrame(random_search.cv_results_)
    cv_results_file = f"{timestamp}_results.csv"
    cv_results.to_csv(os.path.join(results_path, cv_results_file), index=False)
    
    # Save .ipynb
    copy_current_nb(os.path.join(results_path, 'Evaluation_Notebook'))
    
    # Save Model
    file_name = f"xgb_{timestamp}.pkl"
    pickle.dump(random_search, open(os.path.join(results_path, file_name), "wb"))
        

except Exception as e:
    print(f"An error occurred during model optimization: {e}")


Fitting 5 folds for each of 500 candidates, totalling 2500 fits
[CV 2/5; 2/500] START colsample_bylevel=0.6398760460621272, colsample_bynode=0.6604297339327864, colsample_bytree=0.8909285194831833, gamma=0.4561738017712562, learning_rate=0.07795562585902209, max_depth=24, n_estimators=86, reg_alpha=0.010790441341474333, reg_lambda=1.8140258781293448, subsample=0.8589472484759024
[CV 2/5; 2/500] END colsample_bylevel=0.6398760460621272, colsample_bynode=0.6604297339327864, colsample_bytree=0.8909285194831833, gamma=0.4561738017712562, learning_rate=0.07795562585902209, max_depth=24, n_estimators=86, reg_alpha=0.010790441341474333, reg_lambda=1.8140258781293448, subsample=0.8589472484759024;, score=(train=0.997, test=0.803) total time=   0.7s
[CV 4/5; 3/500] START colsample_bylevel=0.7847967251785161, colsample_bynode=0.6626516517825252, colsample_bytree=0.922542827216404, gamma=0.2575271803657049, learning_rate=0.08655512044711312, max_depth=39, n_estimators=140, reg_alpha=1.97621676248

[CV 1/5; 1/500] START colsample_bylevel=0.9568623721184107, colsample_bynode=0.9675192403964803, colsample_bytree=0.7438389587386695, gamma=1.850842060817991, learning_rate=0.07355647037225387, max_depth=30, n_estimators=65, reg_alpha=1.5015903556461276, reg_lambda=1.7995961878471345, subsample=0.9897003095921357
[CV 1/5; 1/500] END colsample_bylevel=0.9568623721184107, colsample_bynode=0.9675192403964803, colsample_bytree=0.7438389587386695, gamma=1.850842060817991, learning_rate=0.07355647037225387, max_depth=30, n_estimators=65, reg_alpha=1.5015903556461276, reg_lambda=1.7995961878471345, subsample=0.9897003095921357;, score=(train=0.905, test=0.798) total time=   0.4s
[CV 2/5; 3/500] START colsample_bylevel=0.7847967251785161, colsample_bynode=0.6626516517825252, colsample_bytree=0.922542827216404, gamma=0.2575271803657049, learning_rate=0.08655512044711312, max_depth=39, n_estimators=140, reg_alpha=1.9762167624875888, reg_lambda=0.06384409799692592, subsample=0.6868162676021907
[C

[CV 2/5; 1/500] START colsample_bylevel=0.9568623721184107, colsample_bynode=0.9675192403964803, colsample_bytree=0.7438389587386695, gamma=1.850842060817991, learning_rate=0.07355647037225387, max_depth=30, n_estimators=65, reg_alpha=1.5015903556461276, reg_lambda=1.7995961878471345, subsample=0.9897003095921357
[CV 2/5; 1/500] END colsample_bylevel=0.9568623721184107, colsample_bynode=0.9675192403964803, colsample_bytree=0.7438389587386695, gamma=1.850842060817991, learning_rate=0.07355647037225387, max_depth=30, n_estimators=65, reg_alpha=1.5015903556461276, reg_lambda=1.7995961878471345, subsample=0.9897003095921357;, score=(train=0.910, test=0.807) total time=   0.5s
[CV 3/5; 3/500] START colsample_bylevel=0.7847967251785161, colsample_bynode=0.6626516517825252, colsample_bytree=0.922542827216404, gamma=0.2575271803657049, learning_rate=0.08655512044711312, max_depth=39, n_estimators=140, reg_alpha=1.9762167624875888, reg_lambda=0.06384409799692592, subsample=0.6868162676021907
[C

[CV 1/5; 2/500] START colsample_bylevel=0.6398760460621272, colsample_bynode=0.6604297339327864, colsample_bytree=0.8909285194831833, gamma=0.4561738017712562, learning_rate=0.07795562585902209, max_depth=24, n_estimators=86, reg_alpha=0.010790441341474333, reg_lambda=1.8140258781293448, subsample=0.8589472484759024
[CV 1/5; 2/500] END colsample_bylevel=0.6398760460621272, colsample_bynode=0.6604297339327864, colsample_bytree=0.8909285194831833, gamma=0.4561738017712562, learning_rate=0.07795562585902209, max_depth=24, n_estimators=86, reg_alpha=0.010790441341474333, reg_lambda=1.8140258781293448, subsample=0.8589472484759024;, score=(train=0.999, test=0.808) total time=   0.8s
[CV 5/5; 3/500] START colsample_bylevel=0.7847967251785161, colsample_bynode=0.6626516517825252, colsample_bytree=0.922542827216404, gamma=0.2575271803657049, learning_rate=0.08655512044711312, max_depth=39, n_estimators=140, reg_alpha=1.9762167624875888, reg_lambda=0.06384409799692592, subsample=0.6868162676021

[CV 2/5; 36/500] START colsample_bylevel=0.7086150588478461, colsample_bynode=0.9341157407916101, colsample_bytree=0.7062256458235723, gamma=1.1258743160663542, learning_rate=0.08117478432038895, max_depth=44, n_estimators=124, reg_alpha=1.0353303115188224, reg_lambda=1.7544976288320286, subsample=0.9415046135965655
[CV 2/5; 36/500] END colsample_bylevel=0.7086150588478461, colsample_bynode=0.9341157407916101, colsample_bytree=0.7062256458235723, gamma=1.1258743160663542, learning_rate=0.08117478432038895, max_depth=44, n_estimators=124, reg_alpha=1.0353303115188224, reg_lambda=1.7544976288320286, subsample=0.9415046135965655;, score=(train=0.955, test=0.810) total time=   0.5s
[CV 5/5; 37/500] START colsample_bylevel=0.7429053327961056, colsample_bynode=0.7015250604611424, colsample_bytree=0.8187127049444709, gamma=1.9442478980963538, learning_rate=0.051764122594187736, max_depth=30, n_estimators=92, reg_alpha=0.04113185720171986, reg_lambda=1.9538137911947664, subsample=0.99745659672

...

[CV 3/5; 180/500] START colsample_bylevel=0.8418779163708849, colsample_bynode=0.9492707412962511, colsample_bytree=0.6004761927565422, gamma=0.665231297708192, learning_rate=0.09219934446016165, max_depth=36, n_estimators=143, reg_alpha=1.7707096402508509, reg_lambda=1.1290272556919185, subsample=0.8958387704991233
[CV 3/5; 180/500] END colsample_bylevel=0.8418779163708849, colsample_bynode=0.9492707412962511, colsample_bytree=0.6004761927565422, gamma=0.665231297708192, learning_rate=0.09219934446016165, max_depth=36, n_estimators=143, reg_alpha=1.7707096402508509, reg_lambda=1.1290272556919185, subsample=0.8958387704991233;, score=(train=0.960, test=0.865) total time=   0.5s
[CV 1/5; 182/500] START colsample_bylevel=0.7864717139644052, colsample_bynode=0.6342106670750735, colsample_bytree=0.6535033289457751, gamma=1.379800615309562, learning_rate=0.07533539068960743, max_depth=38, n_estimators=129, reg_alpha=1.7523663783889547, reg_lambda=0.49346934787958596, subsample=0.90366074607

[CV 1/5; 186/500] END colsample_bylevel=0.638771506921351, colsample_bynode=0.9878044389084039, colsample_bytree=0.972443613061869, gamma=0.9864108040321056, learning_rate=0.07627093478516055, max_depth=38, n_estimators=61, reg_alpha=0.8060054128554563, reg_lambda=0.8419833997044317, subsample=0.6301949998061146;, score=(train=0.952, test=0.789) total time=   0.4s
[CV 4/5; 187/500] START colsample_bylevel=0.9060107643152893, colsample_bynode=0.7105637582095913, colsample_bytree=0.8374436658149662, gamma=0.8683895591105961, learning_rate=0.05950687082797343, max_depth=38, n_estimators=78, reg_alpha=1.163160148024224, reg_lambda=1.477713896496085, subsample=0.8466550289591769
[CV 4/5; 187/500] END colsample_bylevel=0.9060107643152893, colsample_bynode=0.7105637582095913, colsample_bytree=0.8374436658149662, gamma=0.8683895591105961, learning_rate=0.05950687082797343, max_depth=38, n_estimators=78, reg_alpha=1.163160148024224, reg_lambda=1.477713896496085, subsample=0.8466550289591769;, s

[CV 4/5; 186/500] START colsample_bylevel=0.638771506921351, colsample_bynode=0.9878044389084039, colsample_bytree=0.972443613061869, gamma=0.9864108040321056, learning_rate=0.07627093478516055, max_depth=38, n_estimators=61, reg_alpha=0.8060054128554563, reg_lambda=0.8419833997044317, subsample=0.6301949998061146
[CV 4/5; 186/500] END colsample_bylevel=0.638771506921351, colsample_bynode=0.9878044389084039, colsample_bytree=0.972443613061869, gamma=0.9864108040321056, learning_rate=0.07627093478516055, max_depth=38, n_estimators=61, reg_alpha=0.8060054128554563, reg_lambda=0.8419833997044317, subsample=0.6301949998061146;, score=(train=0.950, test=0.798) total time=   0.5s
[CV 2/5; 188/500] START colsample_bylevel=0.627838789807344, colsample_bynode=0.9994265349331235, colsample_bytree=0.6373510690481731, gamma=1.585457967192495, learning_rate=0.08217566751414102, max_depth=49, n_estimators=105, reg_alpha=0.01445109240177711, reg_lambda=0.8017715255778679, subsample=0.9944037836918367

[CV 2/5; 185/500] START colsample_bylevel=0.6552297264328412, colsample_bynode=0.7603335670280968, colsample_bytree=0.6922365703255811, gamma=1.150620950056974, learning_rate=0.09382494647471123, max_depth=38, n_estimators=87, reg_alpha=1.681725832266658, reg_lambda=0.129575434604843, subsample=0.9712539383979314
[CV 2/5; 185/500] END colsample_bylevel=0.6552297264328412, colsample_bynode=0.7603335670280968, colsample_bytree=0.6922365703255811, gamma=1.150620950056974, learning_rate=0.09382494647471123, max_depth=38, n_estimators=87, reg_alpha=1.681725832266658, reg_lambda=0.129575434604843, subsample=0.9712539383979314;, score=(train=0.939, test=0.807) total time=   0.3s
[CV 4/5; 185/500] START colsample_bylevel=0.6552297264328412, colsample_bynode=0.7603335670280968, colsample_bytree=0.6922365703255811, gamma=1.150620950056974, learning_rate=0.09382494647471123, max_depth=38, n_estimators=87, reg_alpha=1.681725832266658, reg_lambda=0.129575434604843, subsample=0.9712539383979314
[CV 

[CV 1/5; 189/500] START colsample_bylevel=0.8345613993604226, colsample_bynode=0.8096305249022304, colsample_bytree=0.7510366471125143, gamma=1.8593882749077053, learning_rate=0.11164953481454304, max_depth=22, n_estimators=127, reg_alpha=1.1039273297862957, reg_lambda=0.8080214252295805, subsample=0.6476557906123913
[CV 1/5; 189/500] END colsample_bylevel=0.8345613993604226, colsample_bynode=0.8096305249022304, colsample_bytree=0.7510366471125143, gamma=1.8593882749077053, learning_rate=0.11164953481454304, max_depth=22, n_estimators=127, reg_alpha=1.1039273297862957, reg_lambda=0.8080214252295805, subsample=0.6476557906123913;, score=(train=0.922, test=0.801) total time=   0.3s
[CV 1/5; 190/500] START colsample_bylevel=0.8651412035031776, colsample_bynode=0.9302956095071913, colsample_bytree=0.8200195071390025, gamma=0.018222869660705854, learning_rate=0.06044526574719425, max_depth=48, n_estimators=93, reg_alpha=1.4020227662549396, reg_lambda=1.331596231155463, subsample=0.845753961

In [ ]:
cv_results.head()

In [ ]:
results_path = f"./tuning_results/tuning_xgb/{timestamp}/Assets"
if not os.path.exists(results_path):
    os.makedirs(results_path)

# CV Evaluation

In [ ]:
cv_results.columns

In [ ]:
cv_results.sort_values(by='rank_test_score', ascending=True).head(5)

In [ ]:
sorted_cv = cv_results.sort_values(by='rank_test_score', ascending=True)

# Train vs Test Comparison

In [ ]:
plt.figure(figsize=(16, 6))    

plt.plot(sorted_cv['rank_test_score'], sorted_cv['mean_train_score'], label="Train Score")
plt.plot(sorted_cv['rank_test_score'], sorted_cv['mean_test_score'], label="Validation Score")

plt.grid()
plt.xlabel('Sorted Validation Rank')
plt.ylabel('Accuracy')
plt.title('Train and Test Accuracy by Final Rank')
plt.legend(loc='best')

filename = "test_train_by_rank.png"
plt.savefig(os.path.join(results_path, filename))

plt.show()

In [ ]:
# boxplot algorithm comparison
fig = plt.figure(figsize=(10, 3))
fig.suptitle('Test Accuracy by Rank')

ax = fig.add_subplot(111)

plt.boxplot(sorted_cv.iloc[:10, :][['split0_test_score', 'split1_test_score', 'split2_test_score',
   'split3_test_score', 'split4_test_score']].T)
ax.set_xticklabels(range(1, 11))
ax.set_xlabel("Rank")
ax.set_ylabel("Accuracy")

filename = "test_accuracy_by_rank.png"
plt.savefig(os.path.join(results_path, filename))

plt.show()

In [ ]:
# boxplot algorithm comparison
fig = plt.figure(figsize=(10, 3))
fig.suptitle('Train Accuracy by Rank')

ax = fig.add_subplot(111)

plt.boxplot(sorted_cv.iloc[:10, :][['split0_train_score', 'split1_train_score', 'split2_train_score',
   'split3_train_score', 'split4_train_score']].T)
ax.set_xticklabels(range(1, 11))
ax.set_xlabel("Rank")
ax.set_ylabel("Accuracy")

filename = "train_accuracy_by_rank.png"
plt.savefig(os.path.join(results_path, filename))

plt.show()

In [ ]:
max_params = cv_results.loc[cv_results['rank_test_score'] == 1]
best_params = max_params.params.values[0]

In [ ]:
print(f"Mean Train Accuracy = {max_params['mean_train_score'].values[0]:.2f}")
print(f"Mean Validation Accuracy = {max_params['mean_test_score'].values[0]:.2f}")

In [ ]:
random_search = pickle.load(open(os.path.join(f'./tuning_results/tuning_xgb/{timestamp}/', file_name), "rb"))
model = random_search.best_estimator_

#model = XGBClassifier(**best_params)
#model.fit(X_train, y_train)

y_train_prediction = model.predict(X_train)
y_test_prediction = model.predict(X_test)

In [ ]:
print(f"Train set, Accuracy = {accuracy_score(y_train, y_train_prediction):.2f}")
print(f"Test set, Accuracy = {accuracy_score(y_test, y_test_prediction):.2f}")

In [ ]:
ind = np.argpartition(model.feature_importances_, -20)[-20:]

features = X.columns[ind]
importance = model.feature_importances_[ind]

fig = plt.figure(figsize=(12, 6))
plt.barh(range(len(ind)), importance, align='center')
plt.yticks(range(len(ind)), features)
plt.title('Feature Importance XGB')
plt.grid()

filename = "feature_importance.png"
plt.savefig(os.path.join(results_path, filename))
            
plt.show()

In the context of XGBoost, these scores are often computed based on the following:

- Weight: The number of times a feature appears in a tree across the ensemble of trees.
- Gain: The average gain of a feature when it is used in trees.
- Cover: The average coverage of a feature when it is used in trees.

In [ ]:
# TEST
for col in ['param_colsample_bylevel', 'param_colsample_bynode',
       'param_colsample_bytree', 'param_gamma', 'param_learning_rate',
       'param_max_depth', 'param_n_estimators', 'param_reg_alpha',
       'param_reg_lambda', 'param_subsample']:
    
    plt.figure(figsize=(16, 6))    

    m, b = np.polyfit(list(sorted_cv['mean_test_score'].values), list(sorted_cv[col].values), 1)
    plt.plot(sorted_cv['mean_test_score'], m * sorted_cv['mean_test_score'] + b, c='r', label="Regression Line")
    plt.scatter(sorted_cv['mean_test_score'], sorted_cv[col], label=f"{col} Values")
    
    plt.grid()
    plt.xlabel('Mean Validation Score')
    plt.ylabel('Parameter Value')
    plt.title(col)
    plt.legend(loc='best')

    
    filename = f"{col}_by_rank.png"
    plt.savefig(os.path.join(results_path, filename))
                  
    plt.show()


# Hyperparameter Evaluation

In [ ]:

def plot_parameters(x_values, title):
    
    fig, ax1 = plt.subplots(figsize=(16, 6))
    ax2 = ax1.twinx()

    ax1.scatter(x_values, cv_results['mean_test_score'], label='mean_test_score', c='b')
    #ax2.scatter(x_values, cv_results['std_test_score'], label='std_test_score', c='r')

    m, b = np.polyfit(list(x_values.values), list(cv_results['mean_test_score'].values), 1)
    ax1.plot(x_values, m * x_values + b, c='b')

    m, b = np.polyfit(list(x_values.values), list(cv_results['std_test_score'].values), 1)
    ax2.plot(x_values, m * x_values + b, c='r', label='std_test_score')
    
    ax1.set_title(title)
    ax1.set_xlabel('Parameter Value')
    ax1.set_ylabel('Mean Test Score')
    ax2.set_ylabel('Standard Deviation of Test Score')
    ax1.grid(True)
    
    
    # Combine the legends from both axes
    lines, labels = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax2.legend(lines + lines2, labels + labels2, loc='upper right')

    filename = f"{title}_test_score.png"
    plt.savefig(os.path.join(results_path, filename))
            
    plt.show()



In [ ]:
for param in ['param_colsample_bylevel', 'param_colsample_bynode', 'param_colsample_bytree',
                     'param_gamma', 'param_learning_rate', 'param_max_depth', 'param_n_estimators', 
                     'param_reg_alpha', 'param_reg_lambda', 'param_subsample']:
    x_values = cv_results[param]
    plot_parameters(x_values, param)

# Plotting Evaluation Metrics (Precision, Recall, F1-Score, AUC-ROC):


In [ ]:

# For multiclass classification, you need to binarize the labels
y_true_bin = label_binarize(y_test, classes=np.unique(y_test))
y_score_bin = label_binarize(y_test_prediction, classes=np.unique(y_test_prediction))

auc_roc = roc_auc_score(y_true_bin, y_score_bin, average='macro')

# Plot Precision-Recall curve for each class
precision = dict()
recall = dict()

plt.figure(figsize=(16, 6))    
for i in range(7):
    precision[i], recall[i], _ = precision_recall_curve(y_true_bin[:, i], y_score_bin[:, i])
    plt.plot(recall[i], precision[i], label='Covertype {}'.format(i + 1))

plt.grid()
plt.xlabel('Recall')
plt.ylabel('True Positive Rate / Precision')
plt.title('Precision-Recall Curve')
plt.legend(loc='best')

filename = "precision_recall.png"
plt.savefig(os.path.join(results_path, filename))
            
plt.show()


# Plot AUC-ROC curve for each class
fpr = dict()
tpr = dict()

plt.figure(figsize=(16, 6))    
for i in range(7):
    fpr[i], tpr[i], _ = roc_curve(y_true_bin[:, i], y_score_bin[:, i])
    plt.plot(fpr[i], tpr[i], label='Covertype {}'.format(i + 1))

plt.grid()
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate / Precision')
plt.title('ROC Curve')
plt.legend(loc='best')
            
filename = "roc_curve.png"
plt.savefig(os.path.join(results_path, filename))
            
plt.show()


# Partial Dependence

In [ ]:
# potentially iterate over features (and relation ie 0 to 1)

In [ ]:

features, feature_names = [(0,)], [f"Features #{i}" for i in range(X.shape[1])]
deciles = {0: np.linspace(0, 1, num=5)}

pd_results = partial_dependence(
    model, X, features=1, kind="average", grid_resolution=5)

display = PartialDependenceDisplay(
    [pd_results], features=features, feature_names=feature_names,
    target_idx=0, deciles=deciles
)
display.plot(pdp_lim={1: (-1.38, 0.66)})

plt.grid()
plt.xlabel('Feature Value')
plt.ylabel('Partial Dependence') 
plt.title('Partial Dependence')

filename = "partial_dependence.png"
plt.savefig(os.path.join(results_path, filename))
            
plt.show()


# Confusion Matrix

In [ ]:

class_names = np.unique(y)

np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
titles_options = [
    ("Confusion matrix without normalization", None),
    ("Normalized confusion matrix", "true"),
]
for title, normalize in titles_options:
    disp = ConfusionMatrixDisplay.from_estimator(
        model,
        X_test,
        y_test,
        display_labels=class_names + 1,
        cmap=plt.cm.Blues,
        normalize=normalize,
    )
    disp.ax_.set_title(title)

    png_name = title.lower().replace(" ", "_")
    filename = f"{png_name}.png"
    plt.savefig(os.path.join(results_path, filename))

plt.show()

# Shap Values

In [ ]:
explainer = shap.TreeExplainer(model)
explanation = explainer.shap_values(X_test, check_additivity=False)


In [ ]:
shap.summary_plot(explanation, X_test, plot_type="bar", show=False)

filename = f"shap_summary.png"
plt.savefig(os.path.join(results_path, filename))
plt.close()  

SHAP values show how each feature affects each final prediction, the significance of each feature compared to others, and the model's reliance on the interaction between features.


In [ ]:
# IF WE SCALE: SCALE ONLY TRAIN DATA SET
# BOOSTING ALG:

# - Feature Selection: AUTOMATIC (NON CRITICAL)
# - Scaling: AUTOMATIC
# - Handling Missing Values: AUTOMATIC
# - Outliers: NEED TO DEAL WITH

# KAGGLE Prediction

In [ ]:
test_processed = pd.read_csv('Data/test_engineered.csv')

In [ ]:
test_processed.head()

In [ ]:
test_processed = test_processed.loc[:, [col for col in test_processed if not col.startswith('Cover_Type_')]]
X_kaggle = test_processed.drop(columns=['Aspect_Sector'])
y_kaggle = model.predict(X_kaggle) + 1

In [ ]:
pd.read_csv("Data/Kaggle/full_submission.csv").head()

In [ ]:
test_processed['Cover_Type'] = y_kaggle

In [ ]:
kaggle_submission = test_processed.loc[:, ['Id', 'Cover_Type']]

In [ ]:
kaggle_submission.Cover_Type.value_counts()

In [ ]:
kaggle_submission.to_csv(f'Data/kaggle_submission_{timestamp}.csv', index=False)

In [ ]:
pd.read_csv(f'Data/kaggle_submission_{timestamp}.csv')

In [ ]:
timestamp